In [1]:
#mount
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
import sys
root_dir="/content/gdrive/My Drive/Projects/flower"
sys.path.append(root_dir)

%cd "/content/gdrive/My Drive/Projects/flower/wm"

/content/gdrive/My Drive/Projects/flower/wm


In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import importlib
import tradestats
import pandas as pd
from datetime import datetime

In [5]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz

In [71]:
importlib.reload(tradestats)
from tradestats import *

In [68]:
prices = loaddata_1W('mbk_w.csv')

In [76]:
importlib.reload(tradestats)
from tradestats import *
from tradestats import rose
periods = [1]
dictList = []
colFeat = []
keyList = []
roseDict   = rose(prices, periods)
dictList.append(roseDict.df); colFeat.append('rose'); keyList.append(periods)

In [ ]:
# prices.dropna(inplace=True)
masterFrame = pd.DataFrame(index=prices.index)
masterFrame[['id','year','month','day','date','weekday','open','high','low','close','volume']] = prices[['id','year','month','day','date','weekday','open','high','low','close','volume']]

In [ ]:
for i in range(0,len(dictList)):#iteracja po funkcjach
    for j in keyList[i]: #iteracja po okresach
        for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
            colID = colFeat[i] + str(j) + k
            masterFrame[colID] = dictList[i][j][k]

In [ ]:
# masterFrame.to_csv(sep=';',path_or_buf='../Data/masterFrame.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [ ]:
masterFrame = masterFrame.dropna()

In [16]:
masterFrame.tail(5)

NameError: name 'masterFrame' is not defined

In [ ]:
importlib.reload(tradestats)
from tradestats import *
starttime = datetime.now()
# alltrades = preparetrades_brut_tsl(masterFrame,[1],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],2016,2017)
alltrades = preparetrades_brut_tp(masterFrame,[1],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],2016,2017)
endtime = datetime.now()
print(str(endtime - starttime))

In [ ]:
importlib.reload(tradestats)
from tradestats import *
trades = cleartrades_brut(alltrades.copy(),True)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.002],True)

In [ ]:
#____________________________________________________________________________________________________________________
#
#
alltrades.head(2)

In [ ]:
importlib.reload(tradestats)
from tradestats import *
stats1 = runstats_brut_v3(alltrades)